In [ ]:
%load_ext kamu

In [ ]:
%import_dataset mil.nga.msi.asam --alias piracy

In [ ]:
piracy.printSchema()

In [ ]:
%%sql -o piracy -n 999999
    select 
    reference, 
    cast(event_time as string) as event_time, 
    st_asgeojson(geometry) as geometry,
    subreg,
    hostility,
    --victim_d,
    --descriptio,
    hostilityt,
    hostilit_D,
    victim_l,
    victim_l_D,
    navarea
from piracy

In [ ]:
%%local
import json

# For every row we first combine GeoJson geometry with other columns into a Feature object
# Then we combine all Features into a FeatureCollection
def df_to_geojson(df, geom='geometry', props=None):
    if props is None:
        props = [
            c for c in df.columns
            if c != geom
        ]
    
    return {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "geometry": json.loads(row[geom]),
                "properties": {p: str(row[p]) for p in props}
            }
            for _, row in df.iterrows()
        ]
    }

gj = df_to_geojson(piracy)

In [ ]:
%%local
import os
from mapboxgl.viz import *
from mapboxgl.utils import *

# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

viz = CircleViz(
    gj,
    style='mapbox://styles/mapbox/dark-v10',
    center=(-123.1207, 49.2827),
    zoom=1,
    access_token=token,
    radius=2,
    color_default="rgb(255,0,0)",
    #color_property='current_total_value',
    #color_stops=create_color_stops([1000000, 2000000, 3000000, 5000000, 10000000], colors='YlOrRd'),
    #color_default='rgb(158,202,195)',
    #line_width=0,
    #opacity=1.0,
    #legend_layout='horizontal',
    #legend_key_shape='bar',
    #legend_key_borders_on=False
)

viz.show()